# Визуализация и сравнение метрик SUMO: с агентом 1 vs с агентом 2

Этот ноутбук:

- Загружает сетевые и TLS-метрики из двух сценариев (без агента, с агентом)

- Визуализирует временные ряды и строит сводные сравнения

- Даёт интерактивный просмотр метрик по конкретному светофору

# Импорты

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import plotly.express as px

try:
    import ipywidgets as widgets
    from IPython.display import display
    HAS_WIDGETS = True
except Exception:
    HAS_WIDGETS = False

pd.options.display.width = 140
pd.options.display.max_columns = 50

# Пути

In [2]:
PROJECT_ROOT = Path.cwd().parent

SCENARIOS = {
    "with_agent_1": PROJECT_ROOT / "metrics" / "total_reward_lr01_df099_epd0999_30_20_10_0_100eps_7200steps(l_reward_ 1.5 1.2 0.7 g_reward_ 1 1.0 0.5)",
    "with_agent_2": PROJECT_ROOT / "metrics" / "total_reward_lr01_df099_epd0999_acc_in_rew_30_20_10_0_100eps_7200steps(l_reward_ 1.5 1.2 0.7 g_reward_ 1 1.0 0.5)",
}
NETWORK_FILE = "network_metrics.csv"
TLS_FILE = "tls_metrics.csv"

for name, base in SCENARIOS.items():
    net_p = base / NETWORK_FILE
    tls_p = base / TLS_FILE
    print(f"[{name}] network: {net_p.exists()} -> {net_p}")
    print(f"[{name}] tls:     {tls_p.exists()} -> {tls_p}")

[with_agent_1] network: True -> c:\Users\kurganov\workspace\Smart_city\metrics\total_reward_lr01_df099_epd0999_30_20_10_0_100eps_7200steps(l_reward_ 1.5 1.2 0.7 g_reward_ 1 1.0 0.5)\network_metrics.csv
[with_agent_1] tls:     True -> c:\Users\kurganov\workspace\Smart_city\metrics\total_reward_lr01_df099_epd0999_30_20_10_0_100eps_7200steps(l_reward_ 1.5 1.2 0.7 g_reward_ 1 1.0 0.5)\tls_metrics.csv
[with_agent_2] network: True -> c:\Users\kurganov\workspace\Smart_city\metrics\total_reward_lr01_df099_epd0999_acc_in_rew_30_20_10_0_100eps_7200steps(l_reward_ 1.5 1.2 0.7 g_reward_ 1 1.0 0.5)\network_metrics.csv
[with_agent_2] tls:     True -> c:\Users\kurganov\workspace\Smart_city\metrics\total_reward_lr01_df099_epd0999_acc_in_rew_30_20_10_0_100eps_7200steps(l_reward_ 1.5 1.2 0.7 g_reward_ 1 1.0 0.5)\tls_metrics.csv


# Функции

In [3]:
def load_network_metrics(path: Path, scenario: str) -> pd.DataFrame:
    df = pd.read_csv(path / NETWORK_FILE)
    need_cols = {"step", "time", "active_vehicles", "mean_speed_network", "total_queue_len", "total_waiting_time_snapshot"}
    missing = need_cols - set(df.columns)
    if missing:
        raise ValueError(f"Missing columns in {path/NETWORK_FILE}: {missing}")
    df = df.copy()
    df["scenario"] = scenario
    df["time"] = pd.to_numeric(df["time"], errors="coerce")
    df["step"] = pd.to_numeric(df["step"], errors="coerce").astype("Int64")
    df["time_rounded"] = df["time"].round().astype(int)
    return df

def load_tls_metrics(path: Path, scenario: str) -> pd.DataFrame:
    df = pd.read_csv(path / TLS_FILE)
    need_cols = {"step", "time", "tls_id", "phase_index", "tls_queue_len", "tls_waiting_time_snapshot", "tls_mean_speed"}
    missing = need_cols - set(df.columns)
    if missing:
        raise ValueError(f"Missing columns in {path/TLS_FILE}: {missing}")
    df = df.copy()
    df["scenario"] = scenario
    df["time"] = pd.to_numeric(df["time"], errors="coerce")
    df["step"] = pd.to_numeric(df["step"], errors="coerce").astype("Int64")
    df["time_rounded"] = df["time"].round().astype(int)
    df["tls_id"] = df["tls_id"].astype(str)
    return df

def infer_step_interval(df: pd.DataFrame) -> float:
    s = df.sort_values("time_rounded")["time_rounded"].drop_duplicates().diff().dropna()
    if s.empty:
        return np.nan
    return s.mode().iloc[0] if not s.mode().empty else s.median()


def plot_network_metric(df, metric, title, ytitle):
    fig = px.line(
        df.sort_values(["scenario", "time_rounded"]),
        x="time_rounded", y=metric, color="scenario",
        markers=True,
        title=title
    )
    fig.update_layout(xaxis_title="time (s)", yaxis_title=ytitle,
                      legend_title="Scenario", template="plotly_white")
    fig.show()


def p95(x):
    return np.percentile(x, 95)


def rel_change(base_val, var_val):
    if pd.isna(base_val) or pd.isna(var_val) or base_val == 0:
        return np.nan
    return (var_val - base_val) / base_val * 100.0


def auc_trapz(y, x):
    return np.trapz(y, x)


def plot_tls_agg_metric(df, metric, title, ytitle):
    fig = px.line(
        df.sort_values(["scenario", "time_rounded"]),
        x="time_rounded", y=metric, color="scenario", markers=True, title=title
    )
    fig.update_layout(xaxis_title="time (s)", yaxis_title=ytitle,
                      legend_title="Scenario", template="plotly_white")
    fig.show()


def plot_tls_single(tls_id: str, tls_c: pd.DataFrame):
    df = tls_c[tls_c["tls_id"] == tls_id].sort_values(
        ["scenario", "time_rounded"])
    title_base = f"TLS {tls_id}"
    fig1 = px.line(df, x="time_rounded", y="tls_queue_len", color="scenario",
                   markers=True, title=title_base + " — Queue length")
    fig2 = px.line(df, x="time_rounded", y="tls_waiting_time_snapshot",
                   color="scenario", markers=True, title=title_base + " — Waiting time snapshot")
    fig3 = px.line(df, x="time_rounded", y="tls_mean_speed",
                   color="scenario", markers=True, title=title_base + " — Mean speed")
    for fig in (fig1, fig2, fig3):
        fig.update_layout(xaxis_title="time (s)",
                          template="plotly_white", legend_title="Scenario")
        fig.show()

# Загрузка данных

In [4]:
dfs_net = []
dfs_tls = []

for name, base in SCENARIOS.items():
    dfs_net.append(load_network_metrics(base, name))
    dfs_tls.append(load_tls_metrics(base, name))

net = pd.concat(dfs_net, ignore_index=True)
tls = pd.concat(dfs_tls, ignore_index=True)

print("Network rows:", len(net), "TLS rows:", len(tls))
print("Scenarios:", net["scenario"].unique().tolist())
print("TLS count:", tls["tls_id"].nunique())

Network rows: 720 TLS rows: 15840
Scenarios: ['with_agent_1', 'with_agent_2']
TLS count: 22


In [5]:
times_with_1 = set(net.query("scenario == 'with_agent_1'")
                 ["time_rounded"].unique().tolist())
times_with_2 = set(net.query("scenario == 'with_agent_2'")
                 ["time_rounded"].unique().tolist())
common_times = sorted(times_with_1 & times_with_2)

print(
    f"Common sampled times: {len(common_times)} with_1={len(times_with_1)}, with_1={len(times_with_2)})")

if len(common_times) == 0:
    raise RuntimeError(
        "Нет общих временных отметок между сценариями. Проверьте STEP_INTERVAL и файлы.")

net_c = net[net["time_rounded"].isin(common_times)].copy()
tls_c = tls[tls["time_rounded"].isin(common_times)].copy()

print("Inferred sampling interval (s):", infer_step_interval(net_c))

Common sampled times: 360 with_1=360, with_1=360)
Inferred sampling interval (s): 10.0


# Визуализации (сетевые метрики)

In [6]:
plot_network_metric(net_c, "mean_speed_network", "Mean Speed (network)", "m/s")
plot_network_metric(net_c, "total_queue_len", "Total Queue Length (network)", "vehicles stopped")
plot_network_metric(net_c, "total_waiting_time_snapshot", "Total Waiting Time Snapshot (network)", "seconds")
plot_network_metric(net_c, "active_vehicles", "Active Vehicles (network)", "count")

# Сводное сравнение по сети (средние, медианы, p95 и относительные изменения)

In [7]:
import pandas as pd
import numpy as np

SCENARIO_BASE = "with_agent_1"
SCENARIO_VAR = "with_agent_2"

print("Сценарии, присутствующие в net_c:", net_c["scenario"].unique().tolist())

present = set(net_c["scenario"].unique())
missing = [s for s in (SCENARIO_BASE, SCENARIO_VAR) if s not in present]
if missing:
    raise ValueError(
        f"Отсутствуют данные для сценариев: {missing}. "
        f"Есть: {sorted(present)}. Проверьте пути SCENARIOS и CSV."
    )

metrics = ["mean_speed_network", "total_queue_len",
           "total_waiting_time_snapshot", "active_vehicles"]


p95.__name__ = "p95"

agg = (
    net_c
    .groupby("scenario")[metrics]
    .agg(["mean", "median", p95])
)

agg.columns = [f"{m}_{stat}" for m, stat in agg.columns]

base = agg.loc[SCENARIO_BASE]
var = agg.loc[SCENARIO_VAR]

rows = []
for col in agg.columns:
    rows.append({
        "metric": col,
        f"{SCENARIO_BASE}": base[col],
        f"{SCENARIO_VAR}": var[col],
        "relative_change_%": rel_change(base[col], var[col]),
    })

summary = pd.DataFrame(rows)
display(
    summary
    .sort_values("metric")
)

Сценарии, присутствующие в net_c: ['with_agent_1', 'with_agent_2']


,metric,with_agent_1,with_agent_2,relative_change_%
9,active_vehicles_mean,206.905556,242.011111,16.966947
10,active_vehicles_median,220.000000,258.000000,17.272727
11,active_vehicles_p95,240.000000,312.000000,30.000000
0,mean_speed_network_mean,7.843376,6.754091,-13.887967
1,mean_speed_network_median,7.670790,6.466822,-15.695483
2,mean_speed_network_p95,9.732339,9.291834,-4.526199
3,total_queue_len_mean,62.025000,97.894444,57.830624
4,total_queue_len_median,66.000000,105.000000,59.090909
5,total_queue_len_p95,90.000000,149.000000,65.555556
6,total_waiting_time_snapshot_mean,1350.358333,6100.622222,351.778026


# “Площадь под кривой” (AUC) по времени для метрик сети

In [8]:
base_df = net_c[net_c["scenario"] == "with_agent_1"].sort_values("time_rounded")
var_df  = net_c[net_c["scenario"] == "with_agent_2"].sort_values("time_rounded")

joined = base_df.merge(var_df, on="time_rounded", suffixes=("_base", "_var"))

auc_table = []
for metric in ["mean_speed_network", "total_queue_len", "total_waiting_time_snapshot"]:
    y_base = joined[f"{metric}_base"].values
    y_var  = joined[f"{metric}_var"].values
    x      = joined["time_rounded"].values.astype(float)

    auc_b = auc_trapz(y_base, x)
    auc_v = auc_trapz(y_var, x)
    rel   = (auc_v - auc_b) / auc_b * 100.0 if auc_b != 0 else np.nan
    auc_table.append({"metric": metric, "AUC_with_1": auc_b, "AUC_with_2": auc_v, "relative_change_%": rel})

auc_df = pd.DataFrame(auc_table)
display(auc_df)

C:\Users\kurganov\AppData\Local\Temp\ipykernel_9128\2289149121.py:58: DeprecationWarning:

`trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.



,metric,AUC_with_1,AUC_with_2,relative_change_%
0,mean_speed_network,2.819970e+04,2.428516e+04,-13.881481
1,total_queue_len,2.229000e+05,3.516450e+05,57.759085
2,total_waiting_time_snapshot,4.855575e+06,2.186996e+07,350.409375


# TLS: агрегаты по всем светофорам (сумма очередей/ожидания, средняя скорость)

In [9]:
tls_agg = (tls_c
           .groupby(["scenario", "time_rounded"], as_index=False)
           .agg({
               "tls_queue_len": "sum",
               "tls_waiting_time_snapshot": "sum",
               "tls_mean_speed": "mean"
           }))

plot_tls_agg_metric(tls_agg, "tls_queue_len", "TLS agg: Queue length (sum over TLS)", "vehicles stopped")
plot_tls_agg_metric(tls_agg, "tls_waiting_time_snapshot", "TLS agg: Waiting time snapshot (sum over TLS)", "seconds")
plot_tls_agg_metric(tls_agg, "tls_mean_speed", "TLS agg: Mean speed (avg over TLS)", "m/s")

# TLS: интерактивный выбор светофора (если есть ipywidgets)

In [10]:
tls_ids_sorted = sorted(tls_c["tls_id"].unique().tolist())

if HAS_WIDGETS and len(tls_ids_sorted) > 0:
    dropdown = widgets.Dropdown(options=tls_ids_sorted, description="TLS ID:")
    out = widgets.Output()

    def on_change(change):
        if change["name"] == "value":
            with out:
                out.clear_output(wait=True)
                plot_tls_single(change["new"], tls_c)

    dropdown.observe(on_change, names="value")
    display(dropdown, out)
    if tls_ids_sorted:
        plot_tls_single(tls_ids_sorted[0], tls_c)
else:
    print("ipywidgets не установлен или нет TLS. Установите ipywidgets для интерактивности.")

Dropdown(description='TLS ID:', options=('317267021', '318319948', '318319950', '344715428', '459095577', '459…

Output()

# Краткая текстовая сводка

In [11]:
def summarize_changes(net_df):
    lines = []
    metrics = ["mean_speed_network", "total_queue_len", "total_waiting_time_snapshot"]
    stats = (net_df.groupby("scenario")[metrics].mean()).reset_index()
    base = stats[stats["scenario"] == "with_agent_1"].set_index(pd.Index([0]))
    var  = stats[stats["scenario"] == "with_agent_2"].set_index(pd.Index([0]))

    def pct(base_val, var_val):
        return (var_val - base_val) / base_val * 100.0 if base_val != 0 else np.nan

    for m in metrics:
        b = float(base[m].iloc[0])
        v = float(var[m].iloc[0])
        p = pct(b, v)
        direction = "↑ лучше" if (m == "mean_speed_network" and p > 0) or (m != "mean_speed_network" and p < 0) else "↓ хуже"
        lines.append(f"{m}: {b:.3f} -> {v:.3f} ({p:+.1f}%) {direction}")
    return "\n".join(lines)

print("Сводка по средним метрикам (без -> с агентом):")
print(summarize_changes(net_c))

Сводка по средним метрикам (без -> с агентом):
mean_speed_network: 7.843 -> 6.754 (-13.9%) ↓ хуже
total_queue_len: 62.025 -> 97.894 (+57.8%) ↓ хуже
total_waiting_time_snapshot: 1350.358 -> 6100.622 (+351.8%) ↓ хуже
